<img src="https://hilpisch.com/tpq_logo_bic.png"
     width="30%"
     align="right"
     style="border-radius: 8px;">


# Derivatives Analytics with Python
**&mdash;Part IIIb: Market-Based Valuation (Ch.&nbsp;11&ndash;13)**

&copy; Dr. Yves J. Hilpisch | The Python Quants

<a href="https://tpq.io" target="_blank">tpq.io</a> | <a href="https://linktr.ee/dyjh" target="_blank">linktr.ee/dyjh</a>

<img src="https://hilpisch.com/dawp_cover_small.png" width=30% align=left>


## Part IIIb &mdash; Market-Based Valuation

### Chapter 13 &mdash; Dynamic Delta Hedging

This notebook analyzes dynamic delta hedging strategies for American puts.

- Review benchmark hedging results for American puts in the BSM model.
- Implement delta hedging in the calibrated BCC model using LSM.
- Generate pathwise P&L series for individual hedging paths.
- Study the distribution of discounted hedging errors across many paths.


### 1. Environment and figure configuration

We start by importing the core libraries for this class, printing version
information, and configuring the figure export helper used to generate
P&L histograms for the slide deck.


In [ ]:
import sys  # access basic runtime information
from pathlib import Path  # path handling for data and figure export

import math  # elementary math functions

import numpy as np  # numerical arrays
import pandas as pd  # tabular data handling
import matplotlib as mpl  # matplotlib configuration
import matplotlib.pyplot as plt  # plotting

np.set_printoptions(precision=6, suppress=True)  # compact numeric output

print(sys.version.split()[0])  # Python version string
print("NumPy:", np.__version__)  # NumPy version string
print("pandas:", pd.__version__)  # pandas version string
print("matplotlib:", mpl.__version__)  # matplotlib version string

FIG_SAVE = True  # set to True to export figures as PDFs
FIG_DIR = Path("../figures")  # figure output directory
FIG_DPI = 300  # target resolution for exported figures
FIG_DISPLAY = "svg"  # inline display format: "svg" or "png"

plt.style.use("seaborn-v0_8")  # readable plotting defaults
mpl.rcParams["figure.figsize"] = (8.0, 4.5)  # consistent figure size
mpl.rcParams["axes.grid"] = True  # show a grid for readability
mpl.rcParams["savefig.dpi"] = FIG_DPI  # default export resolution

try:
    from matplotlib_inline.backend_inline import set_matplotlib_formats
    set_matplotlib_formats(FIG_DISPLAY)  # configure inline plot rendering
except Exception:
    pass

if FIG_DISPLAY == "png":
    mpl.rcParams["figure.dpi"] = FIG_DPI  # high-resolution inline plots


def maybe_save(fig, filename):
    # Optionally saves a Matplotlib figure as a PDF file.
    if not FIG_SAVE:
        return
    FIG_DIR.mkdir(parents=True, exist_ok=True)
    path = FIG_DIR / f"{filename}.pdf"
    fig.savefig(path, format="pdf", dpi=FIG_DPI)
    print(f"saved: {path}")


CODE_DIR = Path("..") / "code"  # companion module directory
sys.path.insert(0, str(CODE_DIR.resolve()))  # prefer local modules


### 2. BSM benchmark: dynamic hedging and P&L histogram

We first reproduce the benchmark hedging results for an American put in
the BSM model. The goal is to obtain the distribution of discounted
hedging P&L across many simulated paths.


In [ ]:
from dawp_pIIIb_ch13_bsm_hedging import bsm_lsm_put_value

S0 = 36.0  # initial stock value
K = 40.0  # strike price
T = 1.0  # maturity in years
r = 0.06  # constant short rate
sigma = 0.20  # volatility of stock value
M = 50  # number of time steps
I = 10_000  # number of valuation paths
ds = 0.01  # relative perturbation for delta

# Basis for hedging: LSM valuation and regression coefficients.
V_1, S, ex, rg, h, dt = bsm_lsm_put_value(
    S0 + ds,
    K,
    T,
    r,
    sigma,
    n_steps=M,
    n_paths=I,
    seed=123,
)
V_2, _, _, _, _, _ = bsm_lsm_put_value(
    S0,
    K,
    T,
    r,
    sigma,
    n_steps=M,
    n_paths=I,
    seed=123,
)
del_0 = (V_1 - V_2) / ds  # delta at t=0

print("American put value (LSM, BSM):", V_2)
print("Delta at t=0:", del_0)

runs = min(I, 10_000)  # number of dynamic hedging replications
pl_list = []  # list of discounted hedging P&L values

for run in range(runs):
    p = run  # path index
    delta = np.zeros(M + 1, dtype=float)  # delta path

    for t in range(1, M):
        if ex[t - 1, p] == 0:
            St = S[t, p]  # relevant index level
            diff = (
                np.polyval(rg[t], St + ds)
                - np.polyval(rg[t], St)
            )
            delta[t] = diff / ds  # delta as difference quotient
        else:
            break

    delta[0] = del_0
    t_last = t  # exercise or maturity index

    po = np.zeros(t_last + 1, dtype=float)  # portfolio values
    vt = np.zeros(t_last + 1, dtype=float)  # option values
    po[0] = V_2
    vt[0] = V_2
    bo = V_2 - delta[0] * S0  # initial bond position

    for s in range(1, t_last + 1):
        po[s] = delta[s - 1] * S[s, p] + bo * math.exp(r * dt)
          # portfolio value at step s
        if s < t_last:
            bo = po[s] - delta[s] * S[s, p]
              # updated bond position

    vt[t_last] = h[t_last, p]  # option value equals inner value
    pl = (po[t_last] - vt[t_last]) * math.exp(-r * t_last * dt)
      # discounted hedging P&L
    if (run + 1) % 1000 == 0:
        print("run %5d   p/l %8.3f" % (run + 1, pl))
    pl_list.append(pl)

pl_bsm = np.array(pl_list)

print("\nSummary statistics for BSM hedging P&L")
print("replications:", runs)
print("max:", float(pl_bsm.max()))
print("mean:", float(pl_bsm.mean()))
print("median:", float(np.median(pl_bsm)))
print("min:", float(pl_bsm.min()))

fig, ax = plt.subplots()
ax.hist(pl_bsm, bins=75)
ax.set_xlabel("profit/loss")
ax.set_ylabel("frequency")
ax.set_title("BSM American put dynamic-hedging P&L")
fig.tight_layout()

maybe_save(fig, "dawp_pIIIb_fig03_bsm_hedging_pnl_histogram")


### 3. Dynamic hedging in the calibrated BCC model

We now repeat the hedging experiment in the calibrated BCC model.
The goal is to obtain the distribution of discounted hedging P&L for
an at-the-money American put and to compare it to the BSM benchmark.


In [ ]:
from dawp_pIIIb_ch11_bcc_calibration import load_bcc_parameters
from dawp_pIIIb_ch12_bcc_mcs import american_put_lsm

params = load_bcc_parameters()
S0_bcc = params["S0"]
K_bcc = S0_bcc
T_bcc = 1.0
M_bcc = 50
I_bcc = 10_000
a = 1.0  # perturbation parameter
dis = 0.05  # relative perturbation for delta

ds0 = dis * S0_bcc

# Basis LSM runs for S0 shifted up and down.
params_up = dict(params)
params_up["S0"] = S0_bcc + (2.0 - a) * ds0
V_1_bcc, S_bcc, r_bcc, v_bcc, ex_bcc, rg_bcc, h_bcc, dt_bcc = american_put_lsm(
    params_up,
    K=K_bcc,
    T=T_bcc,
    n_steps=M_bcc,
    n_paths=I_bcc,
    basis_degree=3,
    anti_paths=True,
    moment_matching=True,
    seed=50_000,
)

params_dn = dict(params)
params_dn["S0"] = S0_bcc - a * ds0
V_2_bcc, _, _, _, _, _, _, _ = american_put_lsm(
    params_dn,
    K=K_bcc,
    T=T_bcc,
    n_steps=M_bcc,
    n_paths=I_bcc,
    basis_degree=3,
    anti_paths=True,
    moment_matching=True,
    seed=50_000,
)

delt0 = (V_1_bcc - V_2_bcc) / (2.0 * ds0)
V0_LSM, _, _, _, _, _, _, _ = american_put_lsm(
    params,
    K=K_bcc,
    T=T_bcc,
    n_steps=M_bcc,
    n_paths=I_bcc,
    basis_degree=3,
    anti_paths=True,
    moment_matching=True,
    seed=50_000,
)

print("American put value (LSM, BCC):", V0_LSM)
print("Initial delta (BCC):", delt0)

pl_list_bcc = []
runs_bcc = min(I_bcc, 10_000)

for run in range(runs_bcc):
    p = run  # path index
    po = np.zeros(M_bcc + 1, dtype=float)  # portfolio values
    delt = np.zeros(M_bcc + 1, dtype=float)  # deltas
    delt[0] = delt0
    bo = V0_LSM - delt[0] * S0_bcc  # initial bond position

    for t in range(1, M_bcc + 1):
        if ex_bcc[t, p] == 0:
            df = math.exp((r_bcc[t, p] + r_bcc[t - 1, p]) * 0.5 * dt_bcc)
              # discount factor between t-1 and t
            if t != M_bcc:
                po[t] = delt[t - 1] * S_bcc[t, p] + bo * df
                  # portfolio value at step t
                ds_t = dis * S_bcc[t, p]
                  # local perturbation of index level
                sd_a = S_bcc[t, p] + (2.0 - a) * ds_t
                state_a = [
                    sd_a * v_bcc[t, p] * r_bcc[t, p],
                    sd_a * v_bcc[t, p],
                    sd_a * r_bcc[t, p],
                    v_bcc[t, p] * r_bcc[t, p],
                    sd_a ** 2,
                    v_bcc[t, p] ** 2,
                    r_bcc[t, p] ** 2,
                    sd_a,
                    v_bcc[t, p],
                    r_bcc[t, p],
                    1.0,
                ]
                state_a.reverse()
                V_a = max(0.0, float(np.dot(rg_bcc[t], state_a)))

                sd_b = S_bcc[t, p] - a * ds_t
                state_b = [
                    sd_b * v_bcc[t, p] * r_bcc[t, p],
                    sd_b * v_bcc[t, p],
                    sd_b * r_bcc[t, p],
                    v_bcc[t, p] * r_bcc[t, p],
                    sd_b ** 2,
                    v_bcc[t, p] ** 2,
                    r_bcc[t, p] ** 2,
                    sd_b,
                    v_bcc[t, p],
                    r_bcc[t, p],
                    1.0,
                ]
                state_b.reverse()
                V_b = max(0.0, float(np.dot(rg_bcc[t], state_b)))

                delt[t] = (V_a - V_b) / (2.0 * ds_t)
                  # updated delta
                bo = po[t] - delt[t] * S_bcc[t, p]
                  # updated bond position
            else:
                po[t] = delt[t - 1] * S_bcc[t, p] + bo * df
                delt[t] = 0.0
        else:
            df = math.exp((r_bcc[t, p] + r_bcc[t - 1, p]) * 0.5 * dt_bcc)
            po[t] = delt[t - 1] * S_bcc[t, p] + bo * df
            break

    pl = (po[t] - h_bcc[t, p]) * math.exp(-params["r0"] * t * dt_bcc)
      # discounted hedging P&L (flat short-rate proxy)
    if (run + 1) % 1000 == 0:
        print("run %5d   p/l %8.3f" % (run + 1, pl))
    pl_list_bcc.append(pl)

pl_bcc = np.array(pl_list_bcc)

print("\nSummary statistics for BCC hedging P&L")
print("replications:", runs_bcc)
print("max:", float(pl_bcc.max()))
print("mean:", float(pl_bcc.mean()))
print("median:", float(np.median(pl_bcc)))
print("min:", float(pl_bcc.min()))

fig, ax = plt.subplots()
ax.hist(pl_bcc, bins=75)
ax.set_xlabel("profit/loss")
ax.set_ylabel("frequency")
ax.set_title("BCC American put dynamic-hedging P&L")
fig.tight_layout()

maybe_save(fig, "dawp_pIIIb_fig04_bcc_hedging_pnl_histogram")


<img src="https://hilpisch.com/tpq_logo_bic.png"
     width="30%"
     align="right"
     style="border-radius: 8px;">
